<a href="https://colab.research.google.com/github/srikanthreddy-lectures/react_aiml_aws/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir pretrained_models
! cd pretrained_models && wget "http://vllab1.ucmerced.edu/~yli62/CartoonGAN/pytorch_pth/Hayao_net_G_float.pth"

--2022-01-25 10:20:44--  http://vllab1.ucmerced.edu/~yli62/CartoonGAN/pytorch_pth/Hayao_net_G_float.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44529096 (42M)
Saving to: ‘Hayao_net_G_float.pth’

Hayao_net_G_float.p 100%[===================>]  42.47M  51.4MB/s    in 0.8s    

2022-01-25 10:20:44 (51.4 MB/s) - ‘Hayao_net_G_float.pth’ saved [44529096/44529096]

